In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
pd.set_option('float_format', '{:f}'.format)


In [ ]:
#importing data
df = pd.read_csv('kc_house_data.csv')
df

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
price_corr = df.corr()['price'].sort_values(ascending=False)
print(price_corr)

In [ ]:
df = df.drop(['id'],axis=1)

In [ ]:
df.dtypes

## Cleaning

In [ ]:
#changing question marks to 0.0
#df = df.replace('?', 0.0)
#df.view = df.view.replace(np.nan, 0)
#changing all column object types to floats (except date column)
#df.loc[:, df.columns != 'date'] = df.loc[:,df.columns != 'date'].astype('float')

#changing all 0.0 in sqft_basement column, yr_renovated, and waterfront columns to NaN values
#df['sqft_basement'] = df['sqft_basement'].replace(0.0 , np.nan)
#df['waterfront'] = df.waterfront.replace(0.0, np.nan)
#df['yr_renovated'] =df['yr_renovated'].replace(0.0, np.nan)
#changing date column to datetime values
#df['date'] = pd.to_datetime(df['date'])
#df['yr_sold'] = df['date'].dt.to_period('Y')
#df = df.drop(['id'],axis=1)

## Feature Engineering

In [ ]:
#creating has_basement, has_waterfront, has_renovation columns with True/False values
df['has_waterfront'] = df['waterfront'].notnull().astype('int64')
#df['has_basement'] = df['sqft_basement'].notnull().astype('int64')
#df['has_renovation'] = df['yr_renovated'].notnull().astype('int64')
#creating eff_built column (which updates built year depending on whether it was renovated or not)
#df.loc[df['yr_renovated'].notnull(), 'eff_built'] = 2020 - df['yr_renovated']
#df.loc[df['yr_renovated'].isnull(), 'eff_built'] = 2020 -df['yr_built']
#df.eff_built = df.eff_built.astype('int64')

#correcting data types
##discrete vars as int
df.bedrooms = df.bedrooms.astype('int64')
df.bathrooms = df.bathrooms.astype('int64')
df.floors = df.floors.astype('int64')
#df.zipcode = df.zipcode.astype('int64')
#df.condition = df.condition.astype('int64')
df.grade = df.grade.astype('int64')
df.view = df.view.astype('int64')

#categoricals as obj
#df.zipcode = df.zipcode.astype('object')

#drop pre-processed columns
#df = df.drop(['waterfront'],axis=1)


In [ ]:
df['sqft_total'] = df['sqft_basement'] + df['sqft_living']+df['sqft_lot']+df['sqft_above']

In [ ]:
df = df.drop(['sqft_basement','sqft_living','sqft_lot','sqft_above','waterfront','sqft_lot15','sqft_living15'],axis=1)

In [ ]:
df.describe(include='all')

In [ ]:
df.info()

## Export

In [ ]:
df.to_csv('kc_cleaned_corr.csv',index=False)
df

In [ ]:
df = pd.read_csv('kc_cleaned_corr.csv')

In [ ]:
#Prices should be integers, not floats. Pennies are irreleveant when dealing with high prices

df.price = df.price.astype(np.int)

In [ ]:
df.columns

In [ ]:
# Select columns to plot

cols_to_plot = df[['date', 'price', 'bedrooms', 'bathrooms', 'floors', 'view', 'condition',
       'grade', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'has_waterfront', 'sqft_total']]

In [ ]:
# Function to plot scatterplots

def plot_scatterplots():
    for i in cols_to_plot.columns:
        cat_num = cols_to_plot[i].value_counts()
        print('Graph for {}: Total = {}'.format(i.capitalize(), len(cat_num)))
        sns.scatterplot(x=cat_num.index, y=cat_num)
        plt.xticks(rotation=90)
        plt.show()
        
plot_scatterplots()

In [ ]:
#Price Outliers
df.loc[df.price > 6000000]

In [ ]:
# Only 11 homes over $4mil.
# I think we can safely drop these. 

print(len(df.loc[df.price > 4000000]))
df.loc[df.price > 4000000]

In [ ]:
# Drop homes over $4mil

df = df[df.price < 4000000]
df

In [ ]:
#bedroom outliers
# 11 and 33 bedroom homes are clearly anomalies as they only have 1 listing each. 10 bedroom homes are also very rare with 3 listings. 
# I'll remove all homes with 10, 11, and 33 bedrooms. 

df.bedrooms.value_counts()
# Drop 10, 11, and 33 bedroom homes

In [ ]:
df = df[df.bedrooms < 10]
# Let's examine homes with 0 bedrooms.

In [ ]:
# A house with 0 bedrooms doesn't make sense to me. There are houses with 0 bedrooms and 0 bathrooms but have huge lots? 
# Are these possibly offices? Was the data not entered correctly?
# I'm going to drop all of these rows because I think we'll come up with more accurate models this way. 

df[df.bedrooms == 0]
# Drop houses with 0 bedrooms

In [ ]:
df = df[df.bedrooms > 0]

In [ ]:
df.bathrooms.value_counts()
# A house with no bathrooms? Again, is this possibly some sort of store/office? Still seems strange if that's the case. 
# I'm going to drop the 3 rows.

In [ ]:
df[df.bathrooms == 0]

In [ ]:
df = df[df.bathrooms > 0]

In [ ]:
df.bathrooms.value_counts()

In [ ]:
df = df[df.bathrooms < 6]

In [ ]:
# Looks better now.  
df.bathrooms.value_counts()

In [ ]:
#Sqft_living - outliers
# These rows were dropped in one of the previous steps while removing outliers. 

#df[df.sqft_living > 10000]

In [ ]:
#Sqft lot - outliers
#df.sqft_lot.value_counts()

In [ ]:
#print(len(df[df.sqft_lot > 750000]))

In [ ]:
#df[df.sqft_lot > 750000]

In [ ]:
# Let's drop the 11 rows of homes that have a lot of over 7500000 as they seem like outliers.

#df = df[df.sqft_lot < 750000]

In [ ]:
#sqft_living

#df[df.sqft_living > 8000]

In [ ]:
#df = df[df.sqft_living < 8000]

In [ ]:
#df.columns

In [ ]:
#sqft_above

#len(df[df.sqft_above > 6000])

In [ ]:
#df = df[df.sqft_above < 6000]

In [ ]:
#sqft_basement

#len(df[df.sqft_basement > 3000])

In [ ]:
#df = df[df.sqft_basement < 3000]

In [ ]:
#sqft_total

df[df.sqft_total > 750000]

In [ ]:
df = df[df.sqft_total < 750000]

In [ ]:
len(df[df.sqft_total > 750000])

In [ ]:
df.to_csv('kc_cleaned_after_removingoutliers.csv',index=False)
df